In [25]:
import pandas as pd
import requests
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import datetime

from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import MobileApplicationClient

In [26]:
# This is needed to authenticate us to
import os
%load_ext dotenv
%dotenv
try:
    from requests_oauthlib import OAuth2
except ModuleNotFoundError:
    import sys
    import os

    # I need this because requests_oauth gets installed in a weird place on my system
    sys.path.append('/usr/local/lib/python3.6/site-packages')
    from requests_oauthlib import OAuth2

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [27]:
OAuth2

requests_oauthlib.oauth2_auth.OAuth2

In [28]:
access_token = os.getenv("ACCESS_TOKEN")
client_secret = os.getenv("CLIENT_SECRET")
client_id = os.getenv("CLIENT_ID")
callback_url = os.getenv("CALLBACK_URL")
authorization_uri = os.getenv("AUTHORIZATION_URI")

In [16]:
# %load ~/anaconda3/lib/python3.6/site-packages/requests_oauthlib/oauth2_auth.py# %load 
from __future__ import unicode_literals
from oauthlib.oauth2 import WebApplicationClient, InsecureTransportError
from oauthlib.oauth2 import is_secure_transport
from requests.auth import AuthBase


class OAuth2(AuthBase):
    """Adds proof of authorization (OAuth2 token) to the request."""

    def __init__(self, client_id=None, client=None, token=None):
        """Construct a new OAuth 2 authorization object.

        :param client_id: Client id obtained during registration
        :param client: :class:`oauthlib.oauth2.Client` to be used. Default is
                       WebApplicationClient which is useful for any
                       hosted application but not mobile or desktop.
        :param token: Token dictionary, must include access_token
                      and token_type.
        """
        self._client = client or WebApplicationClient(client_id, token=token)
        if token:
            for k, v in token.items():
                setattr(self._client, k, v)

    def __call__(self, r):
        """Append an OAuth 2 token to the request.

        Note that currently HTTPS is required for all requests. There may be
        a token type that allows for plain HTTP in the future and then this
        should be updated to allow plain HTTP on a white list basis.
        """
        if not is_secure_transport(r.url):
            raise InsecureTransportError()
        r.url, r.headers, r.body = self._client.add_token(r.url,
                http_method=r.method, body=r.body, headers=r.headers)
        return r

In [32]:
scope = ["activity", "heartrate", "location", "nutrition", "profile", "settings", "sleep", "social", "weight"]

client = MobileApplicationClient(client_id)

token = {
    'access_token': access_token,
    'token_type': 'Bearer'
}

fitbit = OAuth2Session(client_id, client=client, scope=scope, token=token)

In [33]:
r = fitbit.get('https://api.fitbit.com/1/user/-/profile.json')

In [35]:
response = r.json()

In [36]:
response

{'user': {'age': 25,
  'ambassador': False,
  'autoStrideEnabled': True,
  'avatar': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_100_square.jpg',
  'avatar150': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_150_square.jpg',
  'avatar640': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_640_square.jpg',
  'averageDailySteps': 7822,
  'clockTimeDisplayFormat': '12hour',
  'corporate': False,
  'corporateAdmin': False,
  'country': 'US',
  'dateOfBirth': '1992-12-11',
  'displayName': 'Brenden R.',
  'displayNameSetting': 'name',
  'distanceUnit': 'en_US',
  'encodedId': '2F5M36',
  'familyGuidanceEnabled': False,
  'features': {'exerciseGoal': True},
  'firstName': 'Brenden',
  'foodsLocale': 'en_US',
  'fullName': 'Brenden Rossin',
  'gender': 'MALE',
  'glucoseUnit': 'en_US',
  'height': 180.4,
  'heightUnit': 'en_US',
  'isChild': False,
  'isGraduationAvailable': False,
  'l

In [101]:
yesterday = datetime.datetime(2018,8,27)

date_list = ['2018-08-27']

for i in range(19):
    date = yesterday - datetime.timedelta(days = 31)
    date_list.append(date.strftime('%Y-%m-%d'))
    yesterday = date

In [102]:
date_list

['2018-08-27',
 '2018-07-27',
 '2018-06-26',
 '2018-05-26',
 '2018-04-25',
 '2018-03-25',
 '2018-02-22',
 '2018-01-22',
 '2017-12-22',
 '2017-11-21',
 '2017-10-21',
 '2017-09-20',
 '2017-08-20',
 '2017-07-20',
 '2017-06-19',
 '2017-05-19',
 '2017-04-18',
 '2017-03-18',
 '2017-02-15',
 '2017-01-15']

In [103]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/steps/date/{value}/1m.json')
    response = r.json()
    steps = response['activities-tracker-steps']
    steps_df = pd.DataFrame(steps)
    if index == 0:
        all_steps_df = steps_df
    else:
        all_steps_df = pd.concat([all_steps_df, steps_df])
    
    
all_steps_df

,dateTime,value
0,2018-07-28,4031
1,2018-07-29,0
2,2018-07-30,0
3,2018-07-31,3985
4,2018-08-01,4173
5,2018-08-02,7469
6,2018-08-03,10066
7,2018-08-04,8927
8,2018-08-05,2260
9,2018-08-06,7036


In [104]:
all_steps_df.to_pickle("Data/fitbit_steps.pkl")